In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
import matplotlib.font_manager
import seaborn as sns
%matplotlib inline
from matplotlib.pyplot import figure
import plotly.express as px 
from scipy import stats

==========================================================================================
============Testing the waters for information

In [ ]:
sha = pd.read_csv('../input/shark-attack-dataset/attacks.csv')


In [ ]:
sha.info();

===========================================================================================================

*Columns this Data set contains:*

In [ ]:
sha.columns

Data set size:

In [ ]:
print('This dataset has ' + str(sha.shape[0]) + ' rows, and ' + str(sha.shape[1]) + ' columns')

What this dataset looks like at a glance:

In [ ]:
sha.head()

Alright, let's make our first graph.
So, a univariate analysis on **how many shark attacks happen every year: throughout the decades**.

- First we grab the data.
- Then we plot it in a simple graph.

In [ ]:
year = dict(sha['Year'].value_counts())
year = dict(sorted(year.items()))
year_cop=dict(year)
type(year_cop)
for x, y in year.items():
    if x<1950:
        del year_cop[x]
    else:
        pass
ax1=plt.plot(year_cop.keys(), year_cop.values(), 'b-')
plt.title('Attacks Happened since 1950')

In [ ]:
#year = dict(sha['Year'].value_counts())
#year = dict(sorted(year.items()))
#year_cop=dict(year)

Years = sha['Year'].value_counts()
Years.describe()

Finding out the time of attack, and analysing it.
Mean, Median, Mode.

In [ ]:
# filling int the unknown values with string "Unknown shark"
sha['Time'] = sha['Time'].fillna('Unknown')

sha['New Time'] = None
time_pattern = r'[0-9]{2}h[0-9]{2}'

# Looping through every row in the dataFrame
for row in range(len(sha)):
    try:
        time = re.search(time_pattern, sha.iat[row, sha.columns.get_loc('Time')]).group()
        sha.iat[row, sha.columns.get_loc('New Time')] = time
    except:
        
        # Putting in the orignial value from the original Date column
        sha.iat[row, sha.columns.get_loc('New Time')] = sha['Time'].iloc[row]


byTime_count = sha['New Time'].value_counts().reset_index().rename(columns={'New Time':'Count','index':'Time'})
byTime_count.head()

In [ ]:
fig = px.pie(data_frame = byTime_count.iloc[1:11,:],
             values = 'Count',
             names = 'Time',
             title = 'Time of Shark Attack',
             color_discrete_sequence=px.colors.qualitative.Set2
             )

fig.update_traces(textposition ='outside',
                  textinfo = 'label+percent',
                  pull = [0.1] * 10)
fig.update_layout(paper_bgcolor='cornsilk',
                  legend_title = 'Time of Incident',
                  uniformtext_minsize=18,
                  uniformtext_mode='hide',
                  font = dict(
                      family = "Courier New, monospace",
                      size = 18,
                      color = 'black'
                  ))

fig.show()

In [ ]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

**Bivariate Analysis on Shark Species****

In [ ]:
prov_activity = sha[sha.Type == 'Provoked'].groupby('Activity')['Activity'].count().sort_values(ascending=False)[:10]

fig = px.bar(prov_activity, x=prov_activity.values, y=prov_activity.index, orientation='h', labels={'index':'','x':'Attack Count'},
            title = 'Provoked Attacks by Activity')
fig.update_layout(height=500, width=800)
fig.show()

Can be used for correlation analysis

In [ ]:
sha.info();

In [ ]:
year = dict(sha['Year'].value_counts())
year = dict(sorted(year.items()))
year_cop=dict(year)
type(year_cop)
for x, y in year.items():
    if x<1950:
        del year_cop[x]
    else:
        pass
ax1=plt.plot(year_cop.keys(), year_cop.values(), 'b-')
plt.title('Attacks Happened since 1950')

In [ ]:
import re

# Filling NaN value with string "Unknown shark"
sha['Species'] = sha['Species'].fillna('Unknown shark')

# Create new column to stored the extracted Shark species
sha['New Species'] = None

# Regular Expression pattern used to extract shark species (ultimately getting everything infront of the word "shark")
shark_pattern = r'.* (shark|Shark)'

# Looping through every row in the dataFrame
for row in range(len(sha)):
    try:
        shark_species = re.search(shark_pattern, sha.iat[row, sha.columns.get_loc('Species')]).group()
        sha.iat[row, sha.columns.get_loc('New Species')] = shark_species
    except:
        # Assuming that if there is nothing before string shark OR doesn't contain string shark at all,
        # then I will put the string Shark involvement not confirmed instead
         sha.iat[row, sha.columns.get_loc('New Species')] = "Shark involvement not confirmed"

bySpecies_count = sha['New Species'].value_counts().reset_index().rename(columns={'New Species':'Count','index':'Species'})
bySpecies_count.head()

In [ ]:
# Visualization - I won't be including first two rows of this dataFrame because the species of Shark is unknwon
fig = px.pie(data_frame = bySpecies_count.iloc[2:12,:],
             values = 'Count',
             names = 'Species',
             title = 'Top 10 Shark Species',
             color_discrete_sequence=px.colors.qualitative.Prism
             )

fig.update_traces(textposition ='outside',
                  textinfo = 'percent', pull = [0.1]*10)
fig.update_layout(paper_bgcolor='cornsilk',
                  legend_title = 'Shark Type',
                  font = dict(
                      family = "Courier New, monospace",
                      size = 18,
                      color = 'black'
                  ))

fig.show()

In [ ]:
import plotly.graph_objects as go

#rename species column
sha = sha.rename(columns={'Species ':'Species'})
# species that attack most
# Not including unnamed species
species_attack = sha.groupby('Species')['Species'].count().sort_values(ascending=False)[1:15]

data = go.Bar(x = species_attack.index,y=species_attack.values,text=species_attack.values,textposition='auto', marker_color='lightblue')

layout = go.Layout(title = 'Shark Attack by Species', 
                   xaxis=dict(title='Species'),
                   yaxis=dict(title='Attack Count',visible=False),
                 paper_bgcolor='rgba(244,216,198,1.00)',
                 plot_bgcolor='rgba(252,248,244,1.00)'
                  )

fig = go.Figure(
    data=data,
    layout=layout
)   
fig.show()

In [ ]:
#Species Attack Mean, Median, Min, Max, STD analysis
species_attack.describe()

In [ ]:
#Data information for species attack
species_attack.head()

In [ ]:
#Provoked activity Mean, Median, Min, Max, STD analysis
prov_activity.describe()

In [ ]:
prov_activity.head()